##### Load scraped data from csv file

In [18]:
import pandas as pd
import scraper
import importlib

importlib.reload(scraper) 

<module 'scraper' from 'c:\\Users\\USER\\Documents\\HouseSigma Scraper\\housesigma-scraper\\scraper.py'>

In [19]:
# Load the DataFrame from the CSV file
input_df = pd.read_csv('filtered_sold_data.csv')
input_df = input_df.drop_duplicates().reset_index(drop=True)

print(input_df.shape)
print(input_df.columns)
print("\n ===================== \n")
print(input_df.dtypes)
print("\n ===================== \n")
print(input_df.head())

(764, 10)
Index(['Asking Price', 'Sold Price', 'Address', 'Unit Type', 'Bedrooms',
       'Bathrooms', 'Postal Code', 'Listing URL', 'Bedroom Category',
       'Postal Code Area'],
      dtype='object')


Asking Price        float64
Sold Price          float64
Address              object
Unit Type            object
Bedrooms             object
Bathrooms             int64
Postal Code          object
Listing URL          object
Bedroom Category     object
Postal Code Area     object
dtype: object


   Asking Price  Sold Price  \
0      998000.0    975000.0   
1     1259000.0   1349000.0   
2     3599900.0   3250000.0   
3      825000.0    780000.0   
4      559000.0    529000.0   

                                             Address        Unit Type  \
0   623 - 88 Colgate Ave , Toronto - South Riverdale        Condo Apt   
1  84A Aylesworth Ave , Scarborough - Birchcliffe...         Detached   
2            501 Avonwood Dr , Mississauga - Mineola         Detached   
3  314 - 199 Pine Gr

##### Extract maintenance fees from urls

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

print("🔄 Reinitializing WebDriver...")

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ Open HouseSigma for manual login
driver.get("https://housesigma.com")
input("🔑 Press Enter after logging in manually...")  # ⬅️ Login manually, then press Enter

input_extracted_data = scraper.process_with_progress(input_df, driver, progress_interval=100)

# ✅ Convert the extracted dictionary output into a DataFrame
input_extracted_df = input_extracted_data.apply(pd.Series)

# ✅ Close the browser after processing all listings
driver.quit()
print(input_extracted_df.shape)
print("\n✅ All listings processed. Browser closed.")

🔄 Reinitializing WebDriver...
🚀 Processing row 1/764
🚀 Processing row 101/764
🚀 Processing row 201/764
🚀 Processing row 301/764
🚀 Processing row 401/764
🚀 Processing row 501/764
🚀 Processing row 601/764
🚀 Processing row 701/764
🚀 Processing row 764/764
(764, 3)

✅ All listings processed. Browser closed.


In [21]:
# Save the DataFrame to a CSV file
input_extracted_df.to_csv('input_extracted_data.csv', index=False)

# Print a confirmation message
print("DataFrame has been saved to 'input_extracted_data.csv'")

DataFrame has been saved to 'input_extracted_data.csv'
